# Week 2 homework: MLFlow for experiment tracking and model management

The goal of this homework is to get familiar with MLflow, the tool for experiment tracking and model management.

## Q1. Install MLflow

In [ ]:
!mlflow --version


## Data

Taken from the page: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page.

Downloading "Green Taxi Trip Records" for January, February and March 2023.

In [ ]:
# Green Taxi Trip Records for January, 2023
# !wget -O data/green_tripdata_2023-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet

# Green Taxi Trip Records for February, 2023
# !wget -O data/green_tripdata_2023-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet

# Green Taxi Trip Records for March, 2023
# !wget -O data/green_tripdata_2023-03.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet

Use the script `preprocess_data.py` from `week2_pipeline` to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

In [ ]:
!python week2_pipeline/preprocess_data.py --raw_data_path data --dest_path ./output

## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

Use the script `train.py` from `week2_pipeline` for this.

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Launcing the UI with `mlflow ui --backend-store-uri sqlite:///mlflow.db`, then running the following cell:

In [ ]:
!python week2_pipeline/train.py

## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In order to launch the tracking server, we'll use the following command: ``.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`


  --default-artifact-root URI     Directory in which to store artifacts for
                                  any new experiments created. For tracking
                                  server backends that rely on SQL, this
                                  option is required in order to store
                                  artifacts. Note that this flag does not
                                  impact already-created experiments with any
                                  previous configuration of an MLflow server
                                  instance. By default, data will be logged to
                                  the mlflow-artifacts:/ uri proxy if the
                                  --serve-artifacts option is enabled.
                                  Otherwise, the default location will be
                                  ./mlruns.


  --serve-artifacts / --no-serve-artifacts
                                  Enables serving of artifact uploads,
                                  downloads, and list requests by routing
                                  these requests to the storage location that
                                  is specified by '--artifacts-destination'
                                  directly through a proxy. The default
                                  location that these requests are served from
                                  is a local './mlartifacts' directory which
                                  can be overridden via the '--artifacts-
                                  destination' argument. To disable artifact
                                  serving, specify `--no-serve-artifacts`.
                                  Default: True


  --artifacts-destination URI     The base artifact location from which to
                                  resolve artifact upload/download/list
                                  requests (e.g. 's3://my-bucket'). Defaults
                                  to a local './mlartifacts' directory. This
                                  option only applies when the tracking server
                                  is configured to stream artifacts and the
                                  experiment's artifact root location is http
                                  or mlflow-artifacts URI.

## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

* 4.817
* 5.335
* 5.818
* 6.336

## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 5.060
* 5.567
* 6.061
* 6.568

## Submit the results

The results are submitted here: https://courses.datatalks.club/mlops-zoomcamp-2025/homework/hw2
